In [2]:
day = '2018-04-12'
format_day = day.replace('-','')
year = format_day[0:4]
month = format_day[4:6]
day = format_day[6:8]
yesterday = '2018-04-11'
topic_month = format_day[0:6]
click_path = 'hdfs://hz-cluster3/user/streams/datastore/original/datacenter_sdk_explore_etl/%s/%s/%s/*/*' %(year,month,day)
today_topic_path = '/user/datacenter/data/headline_ctr/doc_extract4/month=%s/day=%s/*/*.gz,/user/datacenter/data/openplat_ctr/doc_extract/month=%s/day=%s/*/*.gz' %(topic_month,format_day,topic_month,format_day)
old_topic_path = '/user/datacenter/linguangjie/user-profile-history/headline_topic_backup/' + format_day
old_profile_path = 'hdfs://hz-cluster3/user/datacenter/linguangjie/user-profile-history/all-user-profile/%s/*' %(yesterday).replace('-','')


# step 1 prepare doc topic and video topic

In [3]:
old_topic_line =  sc.textFile(old_topic_path, 100)

In [5]:
old_topic_line.count()

3196372

In [12]:
old_topic = old_topic_line.sample(False, 0.01).map(eval)

In [13]:
old_topic.count()

31864

In [14]:
old_topic.first()

{u'TAGS': u'\u89c6\u9891',
 u'boardid': u'video_bbs',
 u'bodySegObj': [],
 u'category': u'\u5386\u53f2/\u53e4\u4ee3\u53f2',
 u'category_v3': u'\u5386\u53f2/\u53e4\u4ee3\u53f2',
 u'commentid': u'NA56SULO050835RB',
 u'continousplay': 0,
 u'create_time': u'',
 u'data_source': u'bjrec',
 u'description': u'\u4e2d\u56fd\u5386\u53f2\u4e0a\u7684\u56db\u4e2a\u519b\u4e8b\u5929\u624d\uff0c\u4ece\u6ca1\u8d25\u8fc7\uff0c\u5374\u90fd\u4e0d\u5f97\u5584\u7ec8\uff01',
 u'detail_category_list': u'\u5386\u53f2_\u53e4\u4ee3\u53f2_NA_\u4e2d\u56fd\u5386\u53f2,\u519b\u4e8b_\u519b\u60c5_NA_\u519b\u4e8b',
 u'digest': u'\u4e2d\u56fd\u5386\u53f2\u4e0a\u7684\u56db\u4e2a\u519b\u4e8b\u5929\u624d\uff0c\u4ece\u6ca1\u8d25\u8fc7\uff0c\u5374\u90fd\u4e0d\u5f97\u5584\u7ec8\uff01',
 u'dkeys': u'\u5434\u8d77 \u5218\u90a6 \u767d\u8d77 \u97e9\u4fe1',
 u'doc_byte_count': 0,
 u'doc_char_count': 0,
 u'doc_segment': [],
 u'doc_word_count': 0,
 u'docid': u'VNA56SULO',
 u'end_time': u'',
 u'expireTime': u'2018-08-19 09:10:29',
 u'e

In [ ]:
def is_doc():
    

In [17]:
def filter_doc_feature(line):
    docid = ''
    topic_json = eval(str(line))
    if 'docid' in topic_json and len(topic_json['docid']) == 16 and topic_json['docid'][8:12].isdigit():
        docid = topic_json['docid'].encode('utf-8')
        yield (docid, topic_json)
    if 'docid' in topic_json and topic_json['docid'].startswith('V'):
        docid = topic_json['docid'].encode('utf-8')
        yield (docid, topic_json)

In [18]:
all_topic_data = old_topic_line.flatMap(filter_doc_feature)

In [19]:
all_topic_data.count()

KeyboardInterrupt: 

# step 2 prepare click data

In [20]:
click_line = sc.textFile(click_path)

In [21]:
click_line.count()

KeyboardInterrupt: 

In [23]:
click_line.first()

u'{"appId":"2x1kfBk63z","column":"\u5934\u6761","devId":"CQk0NjU4YzBjZDgxZDRmMDcyCWZlMjA4N2Q%3D","du":2003,"event":"EV","id":"DF3RQ35N052198I6","ip":"116.4.93.11","logId":"1523462400234","offset":30,"passport":"","pg":0.0,"platform":"android","receiveTime":1523462717616,"t":"1","timestamp":1523462716099,"upTimestamp":1523462717000}'

In [ ]:
click_data = click_line.flatMap(parse_userid_docid).repartition(2000)